## **CONTENT RECOMMENDATION SYSTEM**

## **PROBLEM DEFINITION & OBJECTIVE**
Modern video platforms need to deal with billions of users and have deep engagement. It highlights several key flaws in traditional approaches:
1. Relational Complexity: Standard collaborative filtering treats user-video interactions as independent events, failing to capture this web of behaviors such as following creators or hierarchical category browsing.
2. The Cold-Start Problem: Since traditional models lack access to the historical data that new users or fresh content need, this implies poor initial experiences and user churning.
3. Scalability vs. Accuracy: With growing data, it is computationally infeasible to maintain real-time inference speed without sacrificing recommendation quality.
4. Preference Evolution: Static models cannot catch up with shifting trends or evolving user tastes; therefore, a system is required that adapts itself without disrupting the user experience.

### Tech Stack 
- **Neo4j (Graph Database)** - stores relationships
- **PyTorch Geometric (GNN Framework)** - learns from graph structure
- **FAISS (Vector Search)** - fast similarity search
- **BPR Loss (Bayesian Personalized Ranking)** - training objective

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import HeteroData
from torch_geometric.nn import GCNConv, GATConv
from neo4j import GraphDatabase
from typing import Dict, List, Tuple, Set, Optional
from collections import defaultdict
from datetime import datetime, timedelta
import numpy as np
import json
import faiss

c:\Users\Arnav\Desktop\IIT_Project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Node Types**:

- User - people who watch/like videos
- Video - content (created by users)
- Category - sub-categories (e.g., "Programming", "Music")
- ParentCategory - top-level categories (e.g., "Technology", "Entertainment")

**Relationship Types**:

- WATCHES → User watches Video (with watch_time)
- LIKES → User likes Video
- CREATED_BY → Video created by User
- BELONGS_TO → Video belongs to Category
- INTERESTED_IN → User interested in Category (computed from watches/likes)
- FOLLOWS → User follows another User
- PARENT_OF → Category has ParentCategory
- SIMILAR_TO → Video similar to another Video (computed)

In [ ]:
class RecommendationGraphDB:
   
    def __init__(self, uri: str, user: str, password: str):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
    
    def close(self):
        self.driver.close()
    
    def create_complete_schema(self):
        with self.driver.session() as session:
            
            constraints = [
                "CREATE CONSTRAINT user_id IF NOT EXISTS FOR (u:User) REQUIRE u.user_id IS UNIQUE",
                "CREATE CONSTRAINT video_id IF NOT EXISTS FOR (v:Video) REQUIRE v.video_id IS UNIQUE",
                "CREATE CONSTRAINT category_id IF NOT EXISTS FOR (c:Category) REQUIRE c.category_id IS UNIQUE",
                "CREATE CONSTRAINT parent_category_id IF NOT EXISTS FOR (pc:ParentCategory) REQUIRE pc.parent_category_id IS UNIQUE"
            ]
            
            for constraint in constraints:
                try:
                    session.run(constraint)
                except:
                    pass
            
            indexes = [
                "CREATE INDEX user_created IF NOT EXISTS FOR (u:User) ON (u.created_at)",
                "CREATE INDEX video_categories IF NOT EXISTS FOR (v:Video) ON (v.categories)",
                "CREATE INDEX video_parent_categories IF NOT EXISTS FOR (v:Video) ON (v.parent_categories)",
                "CREATE INDEX video_created IF NOT EXISTS FOR (v:Video) ON (v.created_at)",
                "CREATE INDEX video_creator IF NOT EXISTS FOR (v:Video) ON (v.creator_user_id)",
                "CREATE INDEX category_name IF NOT EXISTS FOR (c:Category) ON (c.name)",
                "CREATE INDEX parent_category_name IF NOT EXISTS FOR (pc:ParentCategory) ON (pc.name)",
                "CREATE INDEX watches_timestamp IF NOT EXISTS FOR ()-[r:WATCHES]-() ON (r.timestamp)",
                "CREATE INDEX likes_timestamp IF NOT EXISTS FOR ()-[r:LIKES]-() ON (r.timestamp)"
            ]
            
            for index in indexes:
                try:
                    session.run(index)
                except:
                    pass
    
    def clear_database(self):
        """Clear entire database"""
        with self.driver.session() as session:
            session.run("MATCH (n) DETACH DELETE n")
    
    # ========================================================================
    # NODE CREATION
    # ========================================================================
    
    def create_user(self, user_id: str, properties: Dict = None):
        """
        Create or update user
        Any user can create content - no special creator flag needed
        """
        with self.driver.session() as session:
            props = properties or {}
            
            query = """
            MERGE (u:User {user_id: $user_id})
            SET u += $properties,
                u.created_at = coalesce(u.created_at, datetime()),
                u.updated_at = datetime()
            RETURN u
            """
            session.run(query, user_id=user_id, properties=props)
    
    def create_parent_category(self, parent_category_id: str, name: str,
                              properties: Dict = None):
        """Create parent category (top-level)"""
        with self.driver.session() as session:
            props = properties or {}
            props['name'] = name
            
            query = """
            MERGE (pc:ParentCategory {parent_category_id: $parent_category_id})
            SET pc += $properties,
                pc.created_at = coalesce(pc.created_at, datetime())
            RETURN pc
            """
            session.run(query, parent_category_id=parent_category_id, properties=props)
    
    def create_category(self, category_id: str, name: str,
                       parent_category_id: str, properties: Dict = None):
        """
        Create category linked to parent category
        No recursion - simple one-level hierarchy
        """
        with self.driver.session() as session:
            props = properties or {}
            props['name'] = name
            props['parent_category_id'] = parent_category_id
            
            query = """
            MERGE (c:Category {category_id: $category_id})
            SET c += $properties,
                c.created_at = coalesce(c.created_at, datetime())
            
            MERGE (pc:ParentCategory {parent_category_id: $parent_category_id})
            MERGE (c)-[:PARENT_OF]->(pc)
            
            RETURN c, pc
            """
            session.run(query, 
                       category_id=category_id,
                       parent_category_id=parent_category_id,
                       properties=props)
    
    def create_video(self, video_id: str, title: str, creator_user_id: str,
                    category_ids: List[str], parent_category_ids: List[str],
                    properties: Dict = None):
        with self.driver.session() as session:
            props = properties or {}
            props['title'] = title
            props['creator_user_id'] = creator_user_id
            
            query = """
            MERGE (v:Video {video_id: $video_id})
            SET v += $properties,
                v.categories = $category_ids,
                v.parent_categories = $parent_category_ids,
                v.created_at = coalesce(v.created_at, datetime()),
                v.updated_at = datetime()
            
            MERGE (creator:User {user_id: $creator_user_id})
            MERGE (v)-[:CREATED_BY]->(creator)
            
            WITH v
            UNWIND $category_ids AS cat_id
            MERGE (c:Category {category_id: cat_id})
            ON CREATE SET c.video_count = 0
            MERGE (v)-[:BELONGS_TO]->(c)
            
            WITH v, c
            MATCH (c)<-[:BELONGS_TO]-(videos:Video)
            WITH v, c, count(DISTINCT videos) AS cnt
            SET c.video_count = cnt
            
            RETURN v
            """
            
            session.run(query,
                       video_id=video_id,
                       creator_user_id=creator_user_id,
                       category_ids=category_ids,
                       parent_category_ids=parent_category_ids,
                       properties=props)
    
    # ========================================================================
    # RELATIONSHIP CREATION
    # ========================================================================
    
    def add_watch(self, user_id: str, video_id: str, watch_time: float = 1.0,
                 timestamp: datetime = None, properties: Dict = None):
        with self.driver.session() as session:
            props = properties or {}
            props['watch_time'] = watch_time
            props['timestamp'] = (timestamp or datetime.now()).isoformat()
            
            query = """
            MATCH (u:User {user_id: $user_id})
            MATCH (v:Video {video_id: $video_id})
            
            MERGE (u)-[w:WATCHES]->(v)
            SET w += $properties
            
            WITH u, v, w
            UNWIND v.categories AS cat_id
            MATCH (c:Category {category_id: cat_id})
            
            MERGE (u)-[i:INTERESTED_IN]->(c)
            ON CREATE SET i.score = $watch_time, i.count = 1, i.created_at = datetime()
            ON MATCH SET i.score = i.score + $watch_time, 
                        i.count = i.count + 1,
                        i.updated_at = datetime()
            
            RETURN w
            """
            
            session.run(query, 
                       user_id = user_id,
                       video_id = video_id,
                       watch_time = watch_time,
                       properties = props)
    
    def add_like(self, user_id: str, video_id: str,
                timestamp: datetime = None, properties: Dict = None):
        with self.driver.session() as session:
            props = properties or {}
            props['timestamp'] = (timestamp or datetime.now()).isoformat()
            
            query = """
            MATCH (u:User {user_id: $user_id})
            MATCH (v:Video {video_id: $video_id})
            
            MERGE (u)-[l:LIKES]->(v)
            SET l += $properties
            
            WITH u, v, l
            UNWIND v.categories AS cat_id
            MATCH (c:Category {category_id: cat_id})
            
            MERGE (u)-[i:INTERESTED_IN]->(c)
            ON CREATE SET i.score = 2.0, i.count = 1, i.created_at = datetime()
            ON MATCH SET i.score = i.score + 2.0, i.count = i.count + 1, i.updated_at = datetime()
            
            RETURN l
            """
            
            session.run(query, user_id=user_id, video_id=video_id, properties=props)
    
    def add_user_follows(self, follower_id: str, followee_id: str,
                        timestamp: datetime = None):
        """User follows another user"""
        with self.driver.session() as session:
            ts = (timestamp or datetime.now()).isoformat()
            
            query = """
            MATCH (u1:User {user_id: $follower_id})
            MATCH (u2:User {user_id: $followee_id})
            
            MERGE (u1)-[f:FOLLOWS]->(u2)
            SET f.timestamp = $timestamp
            
            RETURN f
            """
            
            session.run(query, follower_id=follower_id, 
                       followee_id=followee_id, timestamp=ts)
    
    def add_category_interest(self, user_id: str, category_id: str, 
                             score: float = 1.0):
        """Explicitly mark user interest in category"""
        with self.driver.session() as session:
            query = """
            MATCH (u:User {user_id: $user_id})
            MATCH (c:Category {category_id: $category_id})
            
            MERGE (u)-[i:INTERESTED_IN]->(c)
            ON CREATE SET i.score = $score, i.count = 1, i.explicit = true
            ON MATCH SET i.score = i.score + $score, i.count = i.count + 1
            
            RETURN i
            """
            session.run(query, user_id=user_id, category_id=category_id, score=score)
    
    # ========================================================================
    # COMPUTED RELATIONSHIPS
    # ========================================================================
    
    def compute_video_similarities(self, threshold: float = 0.3):
        """Compute SIMILAR_TO between videos"""
        with self.driver.session() as session:
            print("Computing video similarities...")
            
            query = """
            MATCH (v1:Video), (v2:Video)
            WHERE v1.video_id < v2.video_id
            
            // Category overlap (Jaccard)
            WITH v1, v2,
                 toFloat(size([c IN v1.categories WHERE c IN v2.categories])) /
                 size(v1.categories + [c IN v2.categories WHERE NOT c IN v1.categories]) as cat_sim,
                 // Same creator bonus
                 CASE WHEN v1.creator_user_id = v2.creator_user_id THEN 0.3 ELSE 0.0 END as creator_bonus
            
            WITH v1, v2, cat_sim * 0.7 + creator_bonus AS similarity
            WHERE similarity >= $threshold
            
            MERGE (v1)-[s:SIMILAR_TO]-(v2)
            SET s.similarity = similarity,
                s.updated_at = datetime()
            
            RETURN count(s) AS count
            """
            
            result = session.run(query, threshold=threshold)
            count = result.single()['count']
            print(f"  ✓ Created {count} video similarity relationships\n")
    
    # ========================================================================
    # QUERY UTILITIES
    # ========================================================================
    
    def get_creator_videos(self, creator_user_id: str, limit: int = 20) -> List[Dict]:
        """Get videos created by a user"""
        with self.driver.session() as session:
            query = """
            MATCH (creator:User {user_id: $creator_user_id})<-[:CREATED_BY]-(v:Video)
            
            OPTIONAL MATCH (v)<-[w:WATCHES]-()
            WITH v, count(w) AS watch_count
            ORDER BY watch_count DESC
            LIMIT $limit
            
            RETURN v.video_id AS video_id,
                   v.title AS title,
                   v.categories AS categories,
                   watch_count
            """
            result = session.run(query, creator_user_id=creator_user_id, limit=limit)
            return [dict(record) for record in result]
    
    def get_videos_by_parent_category(self, parent_category_id: str, 
                                     limit: int = 20) -> List[Dict]:
        """
        Get videos by parent category (fast - no recursion!)
        Uses parent_categories property
        """
        with self.driver.session() as session:
            query = """
            MATCH (v:Video)
            WHERE $parent_cat_id IN v.parent_categories
            
            OPTIONAL MATCH (v)<-[w:WATCHES]-()
            WITH v, count(w) AS watch_count
            ORDER BY watch_count DESC
            LIMIT $limit

            RETURN v.video_id AS video_id,
                   v.title AS title,
                   v.categories AS categories,
                   watch_count
            """
            result = session.run(query, parent_cat_id=parent_category_id, limit=limit)
            return [dict(record) for record in result]
    
    def get_user_history(self, user_id: str, limit: int = 50) -> List[Dict]:
        """Get user's watch/like history"""
        with self.driver.session() as session:
            query = """
            MATCH (u:User {user_id: $user_id})-[r:WATCHES|LIKES]->(v:Video)
            RETURN v.video_id AS video_id,
                   v.title AS title,
                   v.categories AS categories,
                   v.parent_categories AS parent_categories,
                   type(r) AS action,
                   r.timestamp AS timestamp,
                   r.watch_time AS watch_time
            ORDER BY r.timestamp DESC
            LIMIT $limit
            """
            result = session.run(query, user_id=user_id, limit=limit)
            return [dict(record) for record in result]
    
    def get_user_interests(self, user_id: str) -> List[Dict]:
        """Get user's category interests"""
        with self.driver.session() as session:
            query = """
            MATCH (u:User {user_id: $user_id})-[i:INTERESTED_IN]->(c:Category)
            OPTIONAL MATCH (c)-[:PARENT_OF]->(pc:ParentCategory)
            
            RETURN c.category_id AS category_id,
                   c.name AS category_name,
                   pc.parent_category_id AS parent_category_id,
                   pc.name AS parent_category_name,
                   i.score AS score,
                   i.count AS interaction_count
            ORDER BY i.score DESC
            """
            result = session.run(query, user_id=user_id)
            return [dict(record) for record in result]
    
    def get_creators_user_follows(self, user_id: str) -> List[str]:
        """
        Get users that this user follows who have created content
        In social media, anyone who creates content is a "creator"
        """
        with self.driver.session() as session:
            query = """
            MATCH (u:User {user_id: $user_id})-[:FOLLOWS]->(creator:User)
            WHERE exists((creator)<-[:CREATED_BY]-(:Video))
            RETURN DISTINCT creator.user_id AS creator_user_id
            """
            result = session.run(query, user_id=user_id)
            return [record['creator_user_id'] for record in result]
    
    # ========================================================================
    # DATA RETRIEVAL FOR GNN
    # ========================================================================
    
    def get_all_nodes(self) -> Dict[str, List[str]]:
        """Get all node IDs by type"""
        with self.driver.session() as session:
            nodes = {}
            
            # Users
            result = session.run("MATCH (u:User) RETURN u.user_id as id")
            nodes['user'] = [r['id'] for r in result]
            
            # Videos
            result = session.run("MATCH (v:Video) RETURN v.video_id as id")
            nodes['video'] = [r['id'] for r in result]
            
            # Categories
            result = session.run("MATCH (c:Category) RETURN c.category_id as id")
            nodes['category'] = [r['id'] for r in result]
            
            # ParentCategories
            result = session.run("MATCH (pc:ParentCategory) RETURN pc.parent_category_id as id")
            nodes['parent_category'] = [r['id'] for r in result]
            
            return nodes
    
    def get_all_edges(self) -> Dict[Tuple[str, str, str], List[Tuple]]:
        """Get all edges for GNN"""
        with self.driver.session() as session:
            edges = {}
            
            # WATCHES
            result = session.run("""
                MATCH (u:User)-[r:WATCHES]->(v:Video)
                RETURN u.user_id, v.video_id, r.watch_time
            """)
            edges[('user', 'watches', 'video')] = [
                (r['u.user_id'], r['v.video_id'], r['r.watch_time'] or 1.0)
                for r in result
            ]
            
            # LIKES
            result = session.run("""
                MATCH (u:User)-[r:LIKES]->(v:Video)
                RETURN u.user_id, v.video_id
            """)
            edges[('user', 'likes', 'video')] = [
                (r['u.user_id'], r['v.video_id'], 1.0) for r in result
            ]
            
            # BELONGS_TO
            result = session.run("""
                MATCH (v:Video)-[r:BELONGS_TO]->(c:Category)
                RETURN v.video_id, c.category_id
            """)
            edges[('video', 'belongs_to', 'category')] = [
                (r['v.video_id'], r['c.category_id'], 1.0) for r in result
            ]
            
            # CREATED_BY (video -> user)
            result = session.run("""
                MATCH (v:Video)-[r:CREATED_BY]->(u:User)
                RETURN v.video_id, u.user_id
            """)
            edges[('video', 'created_by', 'user')] = [
                (r['v.video_id'], r['u.user_id'], 1.0) for r in result
            ]
            
            # FOLLOWS
            result = session.run("""
                MATCH (u1:User)-[r:FOLLOWS]->(u2:User)
                RETURN u1.user_id, u2.user_id
            """)
            edges[('user', 'follows', 'user')] = [
                (r['u1.user_id'], r['u2.user_id'], 1.0) for r in result
            ]
            
            # INTERESTED_IN
            result = session.run("""
                MATCH (u:User)-[r:INTERESTED_IN]->(c:Category)
                RETURN u.user_id, c.category_id, r.score
            """)
            edges[('user', 'interested_in', 'category')] = [
                (r['u.user_id'], r['c.category_id'], min(r['r.score'] / 10.0, 1.0))
                for r in result
            ]
            
            # SIMILAR_TO
            result = session.run("""
                MATCH (v1:Video)-[r:SIMILAR_TO]-(v2:Video)
                WHERE v1.video_id < v2.video_id
                RETURN v1.video_id, v2.video_id, r.similarity
            """)
            edges[('video', 'similar_to', 'video')] = [
                (r['v1.video_id'], r['v2.video_id'], r['r.similarity'])
                for r in result
            ]
            
            # PARENT_OF (category -> parent_category)
            result = session.run("""
                MATCH (c:Category)-[r:PARENT_OF]->(pc:ParentCategory)
                RETURN c.category_id, pc.parent_category_id
            """)
            edges[('category', 'parent_of', 'parent_category')] = [
                (r['c.category_id'], r['pc.parent_category_id'], 1.0)
                for r in result
            ]
            
            return edges
    
    # ========================================================================
    # EMBEDDINGS
    # ========================================================================
    
    def store_embeddings(self, node_type: str, embeddings: Dict[str, np.ndarray],
                        version: str = "v1"):
        """Store embeddings in Neo4j"""
        with self.driver.session() as session:
            if node_type == 'parent_category':
                node_label = 'ParentCategory'
                id_field = 'parent_category_id'
            else:
                node_label = node_type.capitalize()
                id_field = f"{node_type}_id"
            
            count = 0
            for node_id, embedding in embeddings.items():
                emb_str = json.dumps(embedding.tolist())
                
                query = f"""
                MATCH (n:{node_label} {{{id_field}: $node_id}})
                SET n.embedding = $embedding,
                    n.embedding_version = $version,
                    n.embedding_updated = datetime()
                """
                
                session.run(query, node_id=node_id, embedding=emb_str, version=version)
                count += 1
            
            print(f"  ✓ Stored {count} {node_type} embeddings")


### CONVERSION LAYER (PyGConverter)

#### Main Purpose: Transform Neo4j graph into PyTorch Geometric format

In [3]:
class PyGConverter:
    """Convert Neo4j to PyTorch Geometric"""
    
    def __init__(self, db: RecommendationGraphDB):
        self.db = db
        self.node_mappings = {}
        self.reverse_mappings = {}
    
    def build_hetero_data(self) -> HeteroData:
        """Build HeteroData"""
        print("Building PyTorch Geometric HeteroData...")
        
        data = HeteroData()
        nodes = self.db.get_all_nodes()
        
        # Node mappings
        for node_type, node_ids in nodes.items():
            self.node_mappings[node_type] = {nid: idx for idx, nid in enumerate(node_ids)}
            self.reverse_mappings[node_type] = {idx: nid for nid, idx in 
                                                self.node_mappings[node_type].items()}
            
            num_nodes = len(node_ids)
            data[node_type].x = torch.randn(num_nodes, 64)
            data[node_type].num_nodes = num_nodes
            
            print(f"  ✓ {node_type}: {num_nodes} nodes")
        
        # Edges
        edges = self.db.get_all_edges()
        
        for edge_type, edge_list in edges.items():
            if len(edge_list) == 0:
                continue
            
            src_type, rel_type, dst_type = edge_type
            
            edge_index = []
            edge_attr = []
            
            for src_id, dst_id, weight in edge_list:
                if (src_id in self.node_mappings[src_type] and 
                    dst_id in self.node_mappings[dst_type]):
                    
                    src_idx = self.node_mappings[src_type][src_id]
                    dst_idx = self.node_mappings[dst_type][dst_id]
                    
                    edge_index.append([src_idx, dst_idx])
                    edge_attr.append(weight)
            
            if edge_index:
                data[edge_type].edge_index = torch.tensor(edge_index, dtype=torch.long).t()
                data[edge_type].edge_attr = torch.tensor(edge_attr, dtype=torch.float).unsqueeze(1)
                
                print(f"  ✓ {edge_type}: {len(edge_index)} edges")
        
        print("✓ HeteroData created\n")
        return data


### **GNN MODEL**

The GNN learns that:

- Users similar in watching behavior get similar embeddings
- Videos watched by similar users get similar embeddings
- Categories popular with similar users cluster together

#### Main Purpose : To Learn node embeddings from graph structure

```
HIDDEN LAYERS (3 GCN layers with normalization)
For each of 3 layers:
  ├─ GCNConv: x = W * A * x (message passing)
  ├─ LayerNorm: normalize features
  ├─ ReLU: non-linear activation
  ├─ Dropout (0.3): prevent overfitting
  └─ Residual connection: x_new += x_old

MESSAGE PASSING across edge types:
  Each edge type allows information to flow:
  ├─ ('user', 'watches', 'video'): users learn from videos they watch
  ├─ ('video', 'belongs_to', 'category'): videos learn from categories
  ├─ ('user', 'interested_in', 'category'): users learn from categories
  ├─ ('user', 'follows', 'user'): users learn from followed users
  └─ ... all other edge types participate
```

In [4]:
class HeteroGNNModel(nn.Module):
    """Heterogeneous GNN"""
    
    def __init__(self, hidden_dim=128, num_layers=3):
        super().__init__()
        self.hidden_dim = hidden_dim
        
        self.input_projs = nn.ModuleDict({
            'user': nn.Linear(64, hidden_dim),
            'video': nn.Linear(64, hidden_dim),
            'category': nn.Linear(64, hidden_dim),
            'parent_category': nn.Linear(64, hidden_dim)
        })
        
        self.convs = nn.ModuleList([GCNConv(hidden_dim, hidden_dim) for _ in range(num_layers)])
        self.norms = nn.ModuleList([nn.LayerNorm(hidden_dim) for _ in range(num_layers)])
        self.dropout = nn.Dropout(0.3)
    
    def forward(self, x_dict, edge_index_dict):
        for node_type in x_dict.keys():
            x_dict[node_type] = self.input_projs[node_type](x_dict[node_type])
        
        for i, conv in enumerate(self.convs):
            x_dict_new = {}
            
            for edge_type in edge_index_dict.keys():
                src_type, _, dst_type = edge_type
                edge_index = edge_index_dict[edge_type]
                
                if src_type == dst_type:
                    x = conv(x_dict[src_type], edge_index)
                    x = self.norms[i](x)
                    x = F.relu(x)
                    x = self.dropout(x)
                    
                    if dst_type not in x_dict_new:
                        x_dict_new[dst_type] = x
                    else:
                        x_dict_new[dst_type] += x
            
            for node_type in x_dict_new:
                x_dict[node_type] = x_dict[node_type] + x_dict_new[node_type]
        
        return x_dict


### **TRAINING LOOP (RecommenderTrainer)**

#### Main Purpose: Optimize embeddings using Bayesian Personalized Ranking (BPR)

For each user-video pair:
  1. Get positive example: video user actually watched
  2. Sample negative example: random video user didn't watch
  3. Calculate scores:
     - pos_score = user_emb · video_pos_emb (dot product)
     - neg_score = user_emb · video_neg_emb
  4. Loss = -log(sigmoid(pos_score - neg_score))
     (Minimize: we want pos_score >> neg_score)

In [5]:
class RecommenderTrainer:
    """Train GNN with BPR loss"""
    
    def __init__(self, model, data, converter, device='cpu'):
        self.model = model.to(device)
        self.data = data.to(device)
        self.converter = converter
        self.device = device
        self.optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    def train_epoch(self, db):
        self.model.train()
        
        with db.driver.session() as session:
            result = session.run("""
                MATCH (u:User)-[w:WATCHES]->(v:Video)
                RETURN u.user_id, v.video_id
                LIMIT 5000
            """)
            interactions = [(r['u.user_id'], r['v.video_id']) for r in result]
        
        if not interactions:
            return 0.0
        
        total_loss = 0
        num_videos = self.data['video'].num_nodes
        
        for user_id, pos_video_id in interactions:
            u_idx = self.converter.node_mappings['user'].get(user_id)
            v_idx = self.converter.node_mappings['video'].get(pos_video_id)
            
            if u_idx is None or v_idx is None:
                continue
            
            neg_idx = np.random.randint(0, num_videos)
            
            embeddings = self.model(self.data.x_dict, self.data.edge_index_dict)
            
            user_emb = embeddings['user'][u_idx]
            pos_emb = embeddings['video'][v_idx]
            neg_emb = embeddings['video'][neg_idx]
            
            pos_score = (user_emb * pos_emb).sum()
            neg_score = (user_emb * neg_emb).sum()
            
            loss = -F.logsigmoid(pos_score - neg_score)
            
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            
            total_loss += loss.item()
        
        return total_loss / len(interactions)
    
    def train(self, db, num_epochs=30):
        print("Training GNN model...")
        for epoch in range(num_epochs):
            loss = self.train_epoch(db)
            if (epoch + 1) % 10 == 0:
                print(f"  Epoch {epoch+1}/{num_epochs}, Loss: {loss:.4f}")
        print("✓ Training completed\n")
    
    def extract_embeddings(self):
        self.model.eval()
        with torch.no_grad():
            embeddings = self.model(self.data.x_dict, self.data.edge_index_dict)
            result = {}
            for node_type in ['user', 'video', 'category', 'parent_category']:
                result[node_type] = {}
                for node_id, idx in self.converter.node_mappings[node_type].items():
                    result[node_type][node_id] = embeddings[node_type][idx].cpu().numpy()
            return result



### **RECOMMENDATION ENGINE**

#### Main Purpose: Generate personalized recommendations

- Similarity-Based Recommendations
- Creator-Based Recommendations
- Category-Based Recommendations
- Explanation
- COLD START HANDLING
  - Problem: New user has no embeddings
  - Solution: Returns: Most popular videos overall

In [6]:
class RecommendationEngine:
    """Generate recommendations"""
    
    def __init__(self, db, embeddings):
        self.db = db
        self.embeddings = embeddings
        self.faiss_index = None
        self.video_ids = []
        self._build_faiss_index()
    
    def _build_faiss_index(self):
        print("Building FAISS index...")
        video_embeddings = []
        self.video_ids = []
        
        for video_id, emb in self.embeddings['video'].items():
            video_embeddings.append(emb)
            self.video_ids.append(video_id)
        
        video_embeddings = np.array(video_embeddings).astype('float32')
        faiss.normalize_L2(video_embeddings)
        
        dimension = video_embeddings.shape[1]
        self.faiss_index = faiss.IndexFlatIP(dimension)
        self.faiss_index.add(video_embeddings)
        
        print(f"  ✓ FAISS index built with {len(self.video_ids)} videos\n")
    
    def recommend(self, user_id: str, k: int = 10, diversity_weight: float = 0.3):
        """Generate top-K recommendations"""
        user_emb = self.embeddings['user'].get(user_id)
        
        if user_emb is None:
            return self._cold_start_recommend(k)
        
        user_emb = user_emb.reshape(1, -1).astype('float32')
        faiss.normalize_L2(user_emb)
        
        scores, indices = self.faiss_index.search(user_emb, k * 3)
        
        watched = self._get_watched_videos(user_id)
        candidates = [(self.video_ids[idx], float(scores[0][i])) 
                     for i, idx in enumerate(indices[0])
                     if self.video_ids[idx] not in watched]
        
        return self._diversity_rerank(candidates, k, diversity_weight)
    
    def recommend_from_followed_creators(self, user_id: str, k: int = 10):
        """
        Recommend videos from users the current user follows
        (who have created content)
        """
        followed_creators = self.db.get_creators_user_follows(user_id)
        
        if not followed_creators:
            return []
        
        with self.db.driver.session() as session:
            query = """
            MATCH (creator:User)<-[:CREATED_BY]-(v:Video)
            WHERE creator.user_id IN $creator_ids
              AND NOT exists((:User {user_id: $user_id})-[:WATCHES]->(v))
            
            WITH v, creator
            OPTIONAL MATCH (v)<-[w:WATCHES]-()
            WITH v, creator, count(w) as watch_count
            ORDER BY watch_count DESC
            LIMIT $k
            
            RETURN v.video_id as video_id,
                   v.title as title,
                   creator.user_id as creator_id,
                   watch_count
            """
            
            result = session.run(query, 
                               creator_ids=followed_creators,
                               user_id=user_id,
                               k=k)
            return [dict(record) for record in result]
    
    def recommend_by_parent_category(self, parent_category_id: str, 
                                    user_id: Optional[str] = None, k: int = 10):
        """Recommend videos from a parent category"""
        videos = self.db.get_videos_by_parent_category(parent_category_id, limit=k*2)
        
        if user_id:
            watched = self._get_watched_videos(user_id)
            videos = [v for v in videos if v['video_id'] not in watched]
        
        return videos[:k]
    
    def _diversity_rerank(self, candidates, k, lambda_param):
        if len(candidates) <= k:
            return candidates
        
        selected = [candidates[0]]
        candidates = candidates[1:]
        
        while len(selected) < k and candidates:
            best_score = -float('inf')
            best_idx = 0
            
            for i, (video_id, relevance) in enumerate(candidates):
                video_emb = self.embeddings['video'][video_id]
                max_sim = max([np.dot(video_emb, self.embeddings['video'][sel_id])
                              for sel_id, _ in selected])
                mmr_score = lambda_param * relevance - (1 - lambda_param) * max_sim
                
                if mmr_score > best_score:
                    best_score = mmr_score
                    best_idx = i
            
            selected.append(candidates[best_idx])
            candidates.pop(best_idx)
        
        return selected
    
    def _get_watched_videos(self, user_id: str):
        with self.db.driver.session() as session:
            result = session.run("""
                MATCH (u:User {user_id: $user_id})-[:WATCHES]->(v:Video)
                RETURN v.video_id as video_id
            """, user_id=user_id)
            return {r['video_id'] for r in result}
    
    def _cold_start_recommend(self, k: int):
        with self.db.driver.session() as session:
            result = session.run("""
                MATCH (v:Video)<-[w:WATCHES]-()
                WITH v, count(w) as watch_count
                ORDER BY watch_count DESC
                LIMIT $k
                RETURN v.video_id as video_id, watch_count as score
            """, k=k)
            return [(r['video_id'], float(r['score'])) for r in result]
    
    def explain_recommendation(self, user_id: str, video_id: str):
        with self.db.driver.session() as session:
            query = """
            MATCH path = (u:User {user_id: $user_id})-[*1..3]-(v:Video {video_id: $video_id})
            WITH path, length(path) as path_length
            ORDER BY path_length ASC
            LIMIT 3
            RETURN [rel in relationships(path) | type(rel)] as rel_types
            """
            result = session.run(query, user_id=user_id, video_id=video_id)
            
            explanations = []
            for record in result:
                rel_types = record['rel_types']
                if 'INTERESTED_IN' in rel_types:
                    explanations.append("Based on your category interests")
                elif 'FOLLOWS' in rel_types:
                    explanations.append("From creators you follow")
                elif 'CREATED_BY' in rel_types:
                    explanations.append("From a creator whose content you've watched")
                elif 'SIMILAR_TO' in rel_types:
                    explanations.append("Similar to videos you've watched")
            
            return explanations[0] if explanations else "Popular in your interests"


In [7]:
class CompleteRecommendationSystem:
    """Orchestrates the complete pipeline"""
    
    def __init__(self, neo4j_uri: str, neo4j_user: str, neo4j_password: str):
        self.db = RecommendationGraphDB(neo4j_uri, neo4j_user, neo4j_password)
        self.converter = None
        self.data = None
        self.model = None
        self.trainer = None
        self.engine = None
    
    def initialize(self):
        print("="*80)
        print("INITIALIZING RECOMMENDATION SYSTEM")
        print("="*80 + "\n")
        self.db.create_complete_schema()
    
    def load_graph_data(self):
        print("="*80)
        print("LOADING GRAPH DATA")
        print("="*80 + "\n")
        self.converter = PyGConverter(self.db)
        self.data = self.converter.build_hetero_data()
    
    def train_model(self, hidden_dim=128, num_epochs=30):
        print("="*80)
        print("TRAINING GNN MODEL")
        print("="*80 + "\n")
        self.model = HeteroGNNModel(hidden_dim=hidden_dim)
        self.trainer = RecommenderTrainer(self.model, self.data, self.converter)
        self.trainer.train(self.db, num_epochs=num_epochs)
    
    def sync_embeddings(self):
        print("="*80)
        print("SYNCING EMBEDDINGS")
        print("="*80 + "\n")
        embeddings = self.trainer.extract_embeddings()
        for node_type in ['user', 'video', 'category', 'parent_category']:
            self.db.store_embeddings(node_type, embeddings[node_type])
        print("✓ All embeddings synced\n")
        return embeddings
    
    def build_engine(self, embeddings):
        print("="*80)
        print("BUILDING RECOMMENDATION ENGINE")
        print("="*80 + "\n")
        self.engine = RecommendationEngine(self.db, embeddings)
    
    def compute_similarities(self):
        print("="*80)
        print("COMPUTING SIMILARITIES")
        print("="*80 + "\n")
        self.db.compute_video_similarities(threshold=0.3)
    
    def close(self):
        self.db.close()

## **DATA UNDERSTANDING & PREPARATION**
To validate the functioning of the Graph Based Recommendation Engine Synthetic Data is used that is considered to be precisely clean and free from empty or null values but in deployment the data will be gathered from users in real time.

Parent Categories -> Categories '\n',
Synthetically generated random number of users '\n',
Synthesising creators, Videos, interactions and Follow '\n',

```
parent_categories = [
        ('entertainment', 'Entertainment'),
        ('education', 'Education'),
        ('technology', 'Technology'),
        ('lifestyle', 'Lifestyle')
    ]

categories = [
        ('movies', 'Movies', 'entertainment'),
        ('music', 'Music', 'entertainment'),
        ('gaming', 'Gaming', 'entertainment'),
        ('comedy', 'Comedy', 'entertainment'),
        
        ('science', 'Science', 'education'),
        ('history', 'History', 'education'),
        ('math', 'Mathematics', 'education'),
        ('languages', 'Languages', 'education'),
        
        ('programming', 'Programming', 'technology'),
        ('ai', 'Artificial Intelligence', 'technology'),
        ('hardware', 'Hardware', 'technology'),
        ('reviews', 'Tech Reviews', 'technology'),
        
        ('fitness', 'Fitness', 'lifestyle'),
        ('cooking', 'Cooking', 'lifestyle'),
        ('travel', 'Travel', 'lifestyle'),
        ('fashion', 'Fashion', 'lifestyle')
    ]
```

### **Graph Representation**
what exists in Neo4j after populating Sample data

```
Users (100 nodes)
    ├─ WATCHES → Videos (1953 edges)
    ├─ LIKES → Videos (616 edges)
    ├─ FOLLOWS → Users (210 edges)
    └─ INTERESTED_IN → Categories (1466 edges)

Videos (500 nodes)
    ├─ CREATED_BY → Users (500 edges)
    ├─ BELONGS_TO → Categories (1025 edges)
    └─ SIMILAR_TO → Videos (9945 edges)

Categories (16 nodes)
    └─ PARENT_OF → ParentCategories (16 edges)

ParentCategories (4 nodes)
```

In [ ]:
def populate_sample_data(db: RecommendationGraphDB):
    """Populate with sample data"""
    print("="*80)
    print("POPULATING SAMPLE DATA")
    print("="*80 + "\n")
    
    import random
    
    # Parent categories (top-level)
    parent_categories = [
        ('entertainment', 'Entertainment'),
        ('education', 'Education'),
        ('technology', 'Technology'),
        ('lifestyle', 'Lifestyle')
    ]
    
    print("Creating parent categories...")
    for pc_id, name in parent_categories:
        db.create_parent_category(pc_id, name)
    
    # Categories (sub-categories)
    categories = [
        ('movies', 'Movies', 'entertainment'),
        ('music', 'Music', 'entertainment'),
        ('gaming', 'Gaming', 'entertainment'),
        ('comedy', 'Comedy', 'entertainment'),
        
        ('science', 'Science', 'education'),
        ('history', 'History', 'education'),
        ('math', 'Mathematics', 'education'),
        ('languages', 'Languages', 'education'),
        
        ('programming', 'Programming', 'technology'),
        ('ai', 'Artificial Intelligence', 'technology'),
        ('hardware', 'Hardware', 'technology'),
        ('reviews', 'Tech Reviews', 'technology'),
        
        ('fitness', 'Fitness', 'lifestyle'),
        ('cooking', 'Cooking', 'lifestyle'),
        ('travel', 'Travel', 'lifestyle'),
        ('fashion', 'Fashion', 'lifestyle')
    ]
    
    print("Creating categories...")
    for cat_id, name, parent_id in categories:
        db.create_category(cat_id, name, parent_id)
    
    # Create users (any user can create content)
    print("Creating users...")
    users = []
    
    for i in range(100):
        user_id = f"user_{i}"
        db.create_user(user_id, properties={'name': f"User {i}"})
        users.append(user_id)
    
    # Some users will create videos (first 30 users for this example)
    content_creators = users[:30]
    
    # Create videos
    print("Creating videos...")
    videos = []
    category_ids = [c[0] for c in categories]
    
    for i in range(500):
        video_id = f"video_{i}"
        creator = random.choice(content_creators)
        
        # Pick 1-3 categories
        num_cats = random.randint(1, 3)
        selected_cats = random.sample(categories, num_cats)
        cat_ids = [c[0] for c in selected_cats]
        parent_cat_ids = list(set([c[2] for c in selected_cats]))
        
        db.create_video(
            video_id=video_id,
            title=f"Video {i}",
            creator_user_id=creator,
            category_ids=cat_ids,
            parent_category_ids=parent_cat_ids,
            properties={'duration': random.randint(60, 3600)}
        )
        videos.append(video_id)
    
    # Create interactions
    print("Creating interactions...")
    for _ in range(2000):
        user = random.choice(users)
        video = random.choice(videos)
        
        db.add_watch(user, video, 
                    watch_time=random.uniform(0.1, 1.0),
                    timestamp=datetime.now() - timedelta(days=random.randint(0, 90)))
        
        if random.random() > 0.7:
            db.add_like(user, video)
    
    # User follows
    print("Creating social connections...")
    for user in random.sample(users, 50):
        # Follow some content creators
        followed_creators = random.sample(content_creators, random.randint(1, 5))
        for creator in followed_creators:
            if user != creator:
                db.add_user_follows(user, creator)
        
        # Follow some regular users
        followed_users = random.sample(users, random.randint(0, 3))
        for followee in followed_users:
            if user != followee:
                db.add_user_follows(user, followee)
    
    print("✓ Sample data populated\n")

### COMPLETE DATA FLOW

```
┌─────────────────────────────────────────────────────────────┐
│                     INPUT STAGE                             │
├─────────────────────────────────────────────────────────────┤
│  Raw Data (Users, Videos, Interactions)                     │
│              ↓                                              │
│  RecommendationGraphDB.create_complete_schema()             │
│              ↓                                              │
│  Neo4j Database (100 users, 500 videos, relationships)      │
└──────────────────┬──────────────────────────────────────────┘
                   |
┌──────────────────┴──────────────────────────────────────────┐
│                CONVERSION STAGE                             │
├─────────────────────────────────────────────────────────────┤
│  PyGConverter.build_hetero_data()                           │
│              ↓                                              │
│  HeteroData: nodes & edges in PyTorch format                │
│              ↓                                              │
│  Random 64-dim features for all nodes                       │
└──────────────────┬──────────────────────────────────────────┘
                   |
┌──────────────────┴──────────────────────────────────────────┐
│                LEARNING STAGE                               │
├─────────────────────────────────────────────────────────────┤
│  HeteroGNNModel (3 GCN layers)                              │
│              ↓                                              │
│  RecommenderTrainer.train() [30 epochs]                     │
│    - BPR loss: -log(sigmoid(pos - neg))                     │
│    - Optimize: user & video embeddings                      │
│    - Adam optimizer, lr=0.001                               │
│              ↓                                              │
│  Learned embeddings (128-dim for all nodes)                 │
└──────────────────┬──────────────────────────────────────────┘
                   |
┌──────────────────┴──────────────────────────────────────────┐
│              STORAGE & INDEXING STAGE                       │
├─────────────────────────────────────────────────────────────┤
│  Store in Neo4j:                                            │
│    user_0.embedding = [...]                                 │
│    video_0.embedding = [...]                                │
│              ↓                                              │
│  Build FAISS index from video embeddings                    │
│    - 500 × 128 matrix                                       │
│    - L2 normalized                                          │
│    - IndexFlatIP for cosine similarity                      │
└──────────────────┬──────────────────────────────────────────┘
                   | 
┌──────────────────┴──────────────────────────────────────────┐
│            RECOMMENDATION STAGE                             │
├─────────────────────────────────────────────────────────────┤
│  RecommendationEngine                                       │
│              ↓                                              │
│  recommend(user_id):                                        │
│    1. Get user embedding                                    │
│    2. FAISS search for similar videos                       │
│    3. Filter watched & cold-start                           │
│    4. Diversity reranking (MMR)                             │
│    5. Return top-10 with scores                             │
│              ↓                                              │
│  explain_recommendation(user, video):                       │
│    - Query Neo4j for connection paths                       │
│    - Generate explanation                                   │
│              ↓                                              │
│  OUTPUT: [(video_id, score), ...]                           │
└─────────────────────────────────────────────────────────────┘
```

In [ ]:
def main():
    
    # Configuration
    NEO4J_URI = ""
    NEO4J_USER = ""
    NEO4J_PASSWORD = ""
    
    system = CompleteRecommendationSystem(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
    
    try:
        # 1. Initialize
        system.initialize()
        
        # 2. Populate data
        populate_sample_data(system.db)
        
        # 3. Compute similarities
        system.compute_similarities()
        
        # 4. Load graph
        system.load_graph_data()
        
        # 5. Train model
        system.train_model(hidden_dim=128, num_epochs=30)
        
        # 6. Sync embeddings
        embeddings = system.sync_embeddings()
        
        # 7. Build engine
        system.build_engine(embeddings)
        
        # 8. Generate recommendations
        print("="*80)
        print("GENERATING RECOMMENDATIONS")
        print("="*80 + "\n")
        
        test_user = "user_50"
        
        # User history
        print(f"📺 Watch history for {test_user}:")
        history = system.db.get_user_history(test_user, limit=5)
        for item in history:
            print(f"  - {item['action']}: {item['title']}")
            print(f"    Categories: {item['categories']}")
        
        # User interests
        print(f"\n💡 User interests:")
        interests = system.db.get_user_interests(test_user)
        for interest in interests[:5]:
            print(f"  - {interest['category_name']} (parent: {interest['parent_category_name']})")
            print(f"    Score: {interest['score']:.2f}, Interactions: {interest['interaction_count']}")
        
        # General recommendations
        print(f"\n🎯 Top-10 Recommendations:")
        recs = system.engine.recommend(test_user, k=10)
        for i, (video_id, score) in enumerate(recs, 1):
            explanation = system.engine.explain_recommendation(test_user, video_id)
            print(f"  {i}. {video_id} (score: {score:.4f})")
            print(f"     Why? {explanation}")
        
        # Creator-based recommendations
        print(f"\n👥 From users you follow (who create content):")
        creator_recs = system.engine.recommend_from_followed_creators(test_user, k=5)
        for rec in creator_recs:
            print(f"  - {rec['title']} by {rec['creator_id']}")
        
        # Category-based recommendations
        print(f"\n📂 Technology videos:")
        cat_recs = system.engine.recommend_by_parent_category('technology', test_user, k=5)
        for rec in cat_recs:
            print(f"  - {rec['title']} (watches: {rec['watch_count']})")
        
        # Show some user's created content
        creator_user = "user_5"
        print(f"\n🎬 Videos created by {creator_user}:")
        creator_videos = system.db.get_creator_videos(creator_user, limit=5)
        for video in creator_videos:
            print(f"  - {video['title']} (watches: {video['watch_count']})")
        
        print("\n" + "="*80)
        print("✓ PIPELINE COMPLETED SUCCESSFULLY!")
        print("="*80)
        
        print("\n📊 Database Statistics:")
        nodes = system.db.get_all_nodes()
        for node_type, node_list in nodes.items():
            print(f"  {node_type}: {len(node_list)} nodes")
        
        print("\n🚀 Next Steps:")
        print("  1. Neo4j Browser: http://localhost:7474")
        print("  2. Run Cypher queries to explore your graph")
        print("  3. Integrate with your application API")
        print("  4. Set up periodic retraining (daily/weekly)")
        print("  5. Monitor performance and adjust parameters")
        
        print("\n💡 Example Cypher Queries:")
        print("  // Find users who have created content")
        print("  MATCH (u:User)<-[:CREATED_BY]-(v:Video)")
        print("  RETURN u, count(v) as video_count")
        print("")
        print("  // Videos in Entertainment parent category")
        print("  MATCH (v:Video) WHERE 'entertainment' IN v.parent_categories RETURN v")
        print("")
        print("  // User's category interests")
        print("  MATCH (u:User {user_id: 'user_50'})-[i:INTERESTED_IN]->(c:Category)")
        print("  RETURN c.name, i.score ORDER BY i.score DESC")
        print("")
        print("  // Find all content creators a user follows")
        print("  MATCH (u:User {user_id: 'user_50'})-[:FOLLOWS]->(creator:User)")
        print("  WHERE exists((creator)<-[:CREATED_BY]-(:Video))")
        print("  RETURN creator")
        
    except Exception as e:
        print(f"\n❌ Error: {e}")
        import traceback
        traceback.print_exc()
    
    finally:
        system.close()


if __name__ == "__main__":
    main()

INITIALIZING RECOMMENDATION SYSTEM

Creating database schema...
  ✓ user_id
  ✓ video_id
  ✓ category_id
  ✓ parent_category_id
✓ Schema created successfully

POPULATING SAMPLE DATA

Creating parent categories...
Creating categories...
Creating users...
Creating videos...
Creating interactions...
Creating social connections...
✓ Sample data populated

COMPUTING SIMILARITIES

Computing video similarities...
  ✓ Created 9859 video similarity relationships

LOADING GRAPH DATA

Building PyTorch Geometric HeteroData...
  ✓ user: 102 nodes
  ✓ video: 504 nodes
  ✓ category: 41 nodes
  ✓ parent_category: 5 nodes
  ✓ ('user', 'watches', 'video'): 3837 edges
  ✓ ('user', 'likes', 'video'): 1228 edges
  ✓ ('video', 'belongs_to', 'category'): 1933 edges
  ✓ ('video', 'created_by', 'user'): 992 edges
  ✓ ('user', 'follows', 'user'): 424 edges
  ✓ ('user', 'interested_in', 'category'): 1591 edges
  ✓ ('video', 'similar_to', 'video'): 19011 edges
  ✓ ('category', 'parent_of', 'parent_category'): 41 